In [1]:
from numpy import loadtxt
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense 
import pandas as pd 
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD

In [2]:
data = loadtxt('pima-indians-diabetes.csv', delimiter=',')

In [3]:
X = data[:, 0:8]
y = data[:, 8]

In [4]:
ann_model = Sequential()
ann_model.add(Dense(12, input_dim=8, activation='relu'))
ann_model.add(Dense(8, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))

In [5]:
ann_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
ann_model.fit(X, y, epochs=150, batch_size=10, verbose=0)

In [7]:
_, accuracy = ann_model.evaluate(X,y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 1s 8ms/step - loss: 0.5064 - accuracy: 0.7435
Accuracy: 74.35


In [8]:
predictions = ann_model.predict(X)

In [9]:
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)


TUNING THE MODEL TO IMPROVE ACCURACY AND EFFICIENCY
Tuning for the Batch Size and the Number of epochs

In [10]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:

seed=7
np.random.seed(seed)

In [12]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [13]:
batch = [10, 20, 40, 60, 80, 100]
epoch = [10, 50, 100]
parameter_grid = dict(batch_size=batch, epochs=epoch)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [14]:
print('Best %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param, in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

Best 0.694010 using {'batch_size': 100, 'epochs': 100}
0.617188 (0.017758) with: {'batch_size': 10, 'epochs': 10}
0.660156 (0.016877) with: {'batch_size': 10, 'epochs': 50}
0.664062 (0.033754) with: {'batch_size': 10, 'epochs': 100}
0.522135 (0.017566) with: {'batch_size': 20, 'epochs': 10}
0.660156 (0.041463) with: {'batch_size': 20, 'epochs': 50}
0.691406 (0.019918) with: {'batch_size': 20, 'epochs': 100}
0.548177 (0.043420) with: {'batch_size': 40, 'epochs': 10}
0.617188 (0.008438) with: {'batch_size': 40, 'epochs': 50}
0.680990 (0.029635) with: {'batch_size': 40, 'epochs': 100}
0.588542 (0.064133) with: {'batch_size': 60, 'epochs': 10}
0.583333 (0.061045) with: {'batch_size': 60, 'epochs': 50}
0.671875 (0.017758) with: {'batch_size': 60, 'epochs': 100}
0.484375 (0.105445) with: {'batch_size': 80, 'epochs': 10}
0.596354 (0.032264) with: {'batch_size': 80, 'epochs': 50}
0.630208 (0.025780) with: {'batch_size': 80, 'epochs': 100}
0.454427 (0.091628) with: {'batch_size': 100, 'epochs':

In [15]:
def create_model_1(optimizer='adam'):
    model_1 = Sequential()
    model_1.add(Dense(12, input_dim=8, activation='relu'))
    model_1.add(Dense(1, activation='sigmoid'))

    model_1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model_1

Tuning the optimization algorithm 

In [16]:
model_1_1 = KerasClassifier(build_fn=create_model_1, epochs=100, batch_size=10, verbose=0)

In [17]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model_1_1, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [18]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with %r' % (mean, stdev, param))

Best: 0.720052 using {'optimizer': 'Adam'}
0.662760 (0.034104) with {'optimizer': 'SGD'}
0.636719 (0.005524) with {'optimizer': 'RMSprop'}
0.613281 (0.020915) with {'optimizer': 'Adagrad'}
0.630208 (0.024150) with {'optimizer': 'Adadelta'}
0.720052 (0.026557) with {'optimizer': 'Adam'}
0.673177 (0.032578) with {'optimizer': 'Adamax'}
0.688802 (0.012075) with {'optimizer': 'Nadam'}


Tuning the Learning rate and the Momentum

In [19]:

def create_model_2(learn_rate=0.01, momentum=0):
    model_2 = Sequential()
    model_2.add(Dense(12, input_dim=8, activation='relu'))
    model_2.add(Dense(1, activation='sigmoid'))

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model_2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model_2

In [20]:
model_2_2 = KerasClassifier(build_fn=create_model_2, epochs=100, batch_size=10, verbose=0)

In [21]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

In [22]:
param_grid= dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model_2_2, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [23]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip (means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

Best: 0.696615 using {'learn_rate': 0.001, 'momentum': 0.4}
0.651042 (0.012075) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.662760 (0.040133) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.696615 (0.023073) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.671875 (0.025315) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.677083 (0.033502) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.666667 (0.024774) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.666667 (0.009207) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.643229 (0.030145) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.666667 (0.013279) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.656250 (0.030425) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.024774) with: {'learn_rate':

Tuning Network Weight Inittialization

In [24]:
def create_model_3 (init_mode='uniform'):
    model_3 = Sequential()
    model_3.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
    model_3.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))

    model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_3

In [25]:
model_3_3 = KerasClassifier(build_fn=create_model_3, epochs=100, batch_size=10, verbose=0)

In [26]:
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model_3_3, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result.fit(X, y)

GridSearchCV(cv=3,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000002951F578D60>,
             n_jobs=-1,
             param_grid={'learn_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
                         'momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]})

In [27]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param, in zip(means, stds, params):
    print('%f (%f) %r' % (mean, stdev, param))

Best: 0.682292 using {'learn_rate': 0.001, 'momentum': 0.4}
0.640625 (0.019401) {'learn_rate': 0.001, 'momentum': 0.0}
0.674479 (0.033197) {'learn_rate': 0.001, 'momentum': 0.2}
0.682292 (0.020256) {'learn_rate': 0.001, 'momentum': 0.4}
0.682292 (0.021710) {'learn_rate': 0.001, 'momentum': 0.6}
0.677083 (0.029635) {'learn_rate': 0.001, 'momentum': 0.8}
0.651042 (0.021710) {'learn_rate': 0.001, 'momentum': 0.9}
0.664062 (0.005524) {'learn_rate': 0.01, 'momentum': 0.0}
0.651042 (0.024774) {'learn_rate': 0.01, 'momentum': 0.2}
0.644531 (0.022326) {'learn_rate': 0.01, 'momentum': 0.4}
0.661458 (0.038051) {'learn_rate': 0.01, 'momentum': 0.6}
0.651042 (0.024774) {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.024774) {'learn_rate': 0.1, 'momentum': 0.4}
0.651042 (0.024774) {'learn_rate': 0.1, 'momentum': 0.6}
0.649740 (

Tuning the Neuron Activation Function

In [28]:
def create_model_4(activation='relu'):
    model_4 = Sequential()
    model_4.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
    model_4.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

    model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_4

In [29]:
model_4_4 = KerasClassifier(build_fn=create_model_4, epochs=100, batch_size=10, verbose=0)

In [30]:
activation=['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid= dict(activation=activation)
grid = GridSearchCV(estimator=model_4_4, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [31]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with %r' % (mean, stdev, param))

Best: 0.727865 using {'activation': 'softplus'}
0.657552 (0.031948) with {'activation': 'softmax'}
0.727865 (0.015073) with {'activation': 'softplus'}
0.690104 (0.020752) with {'activation': 'softsign'}
0.726562 (0.006379) with {'activation': 'relu'}
0.673177 (0.009744) with {'activation': 'tanh'}
0.675781 (0.024080) with {'activation': 'sigmoid'}
0.691406 (0.012758) with {'activation': 'hard_sigmoid'}
0.718750 (0.028348) with {'activation': 'linear'}


tuning Dropout Regularization

In [32]:
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Dropout

In [33]:
def create_model_5(dropout_rate=0.0, weight_constraint=0):
    model_5 = Sequential()
    model_5.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(weight_constraint)))
    model_5.add(Dropout(dropout_rate))
    model_5.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

    model_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_5

In [34]:
model_5_5 = KerasClassifier(build_fn=create_model_5, epochs=100, batch_size=10, verbose=0)

In [35]:
weight_constraint = [1,2,3,4,5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model_5_5, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [ ]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

means =  grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

Best: 0.717448 using {'dropout_rate': 0.0, 'weight_constraint': 2}
0.696615 (0.015073) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.717448 (0.004872) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.710938 (0.022097) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.710938 (0.008438) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.714844 (0.008438) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.701823 (0.022402) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.712240 (0.038582) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.714844 (0.019137) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.701823 (0.018688) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.717448 (0.003683) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.700521 (0.016367) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.713542 (0.006639) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.710938 (0.008438) with: {'dropout_rate': 0.2, 'weight_constraint': 

Tuning the Number of Neurons on the Hidden layer

In [ ]:
def create_model_6(neurons=1):

    model_6 = Sequential()
    model_6.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(4)))
    model_6.add(Dropout(0.2))
    model_6.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

    model_6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_6

In [ ]:
model_6_6 = KerasClassifier(build_fn=create_model_6, epochs=100, batch_size=10, verbose=0)

In [ ]:
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid =  dict(neurons=neurons)
grid = GridSearchCV(estimator=model_6_6, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result =  grid.fit(X, y)

In [ ]:
print('Best %f using %s' % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

Best parameters:
Batch size: 10 Epoch: 100
optimizer: Adam
learn_rate: 0.001 momentum: 0.0
activation: softplus
dropout_rate: 0.0 weight_constraint: 2
